In [36]:
from easyocr import Reader
import re
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pytesseract
from PIL import Image
from tqdm.auto import tqdm

In [31]:
reader_ch = Reader(['ch_sim'])
common_dict = pd.read_csv('common_chinese_words.csv').set_index('word')['rank'].to_dict()
# Open the video
cap = cv2.VideoCapture('jixiaolan1.mp4')
# Some characteristics from the original video
fps, frames = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)
second = frames/fps

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [112]:
def make_gray(img):
    if np.max(img) <= 1:
        img *= 255
    gray = Image.fromarray((img).astype(np.uint8)).convert('L')
    return np.array(gray)


def dilate_erode(img):
    img = Image.fromarray(img.astype('uint8')).convert('L')
    img = np.array(img)  
    
    kernel = np.ones((2,2), np.uint8)
    img = cv2.dilate(img, kernel, iterations = 4 )
    # Creating kernel
    kernel = np.ones((2, 2), np.uint8)
    # Using cv2.erode() method 
    img = cv2.erode(img, kernel,iterations = 5 )
    
    
    return np.array(img)

def get_x(list_of_lists):
    pos=np.array([np.array(xi) for xi in list_of_lists[0]])
    x = pos[:,0]
    return int(x.min()),int(x.max())

def get_frame(frame_no):
    cap.set(1,frame_no)
    # Here you can define your croping values
    x1,x2,y1,y2 = 935,1015,150,1920-150
    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = cap.read()
    crop_frame = frame[x1:x2,y1:y2,:]
    
    
    ret,crop_frame = cv2.threshold(make_gray(crop_frame),200,255,cv2.THRESH_BINARY_INV)
    return crop_frame

def clean(word):
    word = word.replace('八八','小').replace('和坤','和').replace('和','和珅')
    if len(word)==2 and (word[1] in ['一','二','三'] or word[1] not in common_dict.keys() ):
        word = word[0]
    elif len(word)>=2 and word[-1] in ['一','二','三']:
        word = word[:-1]
    return word

        

In [115]:
def get_text(crop_frame):
    result = reader_ch.readtext(dilate_erode(make_gray(cv2.bitwise_not(crop_frame))),link_threshold=0.99999,width_ths=0,min_size = 75)
    og= reader_ch.readtext(make_gray(cv2.bitwise_not(crop_frame)),link_threshold=0.99999)
    # plt.imshow(dilate_erode(make_gray(cv2.bitwise_not(crop_frame))))

    # old_sent = ''
    # new_sent = ''
    og_sent = ''
    new_results = {}
    old_results = {}
    for i in range(len(result)):
        if '' != result[i][1]:
            x_min, x_max = get_x(result[i])
            word_crop = crop_frame[:,x_min:x_max]
            word = reader_ch.readtext(make_gray(cv2.bitwise_not(word_crop)))
            if len(word)!= 0:
                new_clean_word = clean(re.sub(u'[^\u4E00-\u9FA5]','',' '.join(word[0][1])))

                prob = word[0][-1]
    #             new_sent += new_clean_word
                new_x_min, new_x_max = get_x(word[0])
                new_results[i] = {'pos':[x_min,x_max],'word':new_clean_word,'prob':prob}
            old_clean_word = clean(re.sub(u'[^\u4E00-\u9FA5]','',' '.join(result[i][1])))
    #         old_sent += old_clean_word
            old_results[i] =  {'pos':[x_min,x_max],'word':old_clean_word,'prob':result[i][-1]}
    for i in range(len(og)):     
        og_sent += re.sub(u'[^\u4E00-\u9FA5]','',' '.join(og[i][1]))

    sentence = ''
    for i in range(len(new_results)):
        if i in new_results.keys():
            if new_results[i]['word'] == old_results[i]['word']:
                sentence+= new_results[i]['word']
            elif new_results[i]['word'] != old_results[i]['word'] and new_results[i]['word'] in og_sent:
                sentence+= new_results[i]['word']
            elif new_results[i]['word'] != old_results[i]['word'] and old_results[i]['word'] in og_sent:
                sentence+= old_results[i]['word']
            elif new_results[i]['prob'] > old_results[i]['prob'] + 0.01:
                sentence+= new_results[i]['word']
            elif new_results[i]['prob'] < old_results[i]['prob'] + 0.01:
                sentence+= old_results[i]['word']
            else:
                new_rank = common_dict[new_results[i]['word']]
                old_rank = common_dict[old_results[i]['word']]
                if new_rank < old_rank:
                    sentence += new_results[i]['word']
                else:
                    sentence += old_results[i]['word']
        else:
            try:
                sentence+=old_results[i]['word']
            except:
                pass
    return sentence

In [121]:
srt = ''
for i in tqdm(range(130 + 60+286+31,int(second)-130)):
    crop_frame = get_frame(i * fps)
    text = get_text(crop_frame)
    if len(text) != 0:
        srt+= f'{i}' + text + '\n'

  2%|▉                                           | 45/2068 [01:09<52:04,  1.54s/it]


KeyboardInterrupt: 

In [122]:
print(srt)

550大爷恕咱眼这这哪出啊
551大爷恕咱眼拙这这哪出啊



In [49]:
crop_frame = get_frame(121 * fps)
text = get_text(crop_frame)

In [50]:
text

''